## Setup and Libraries

In [ ]:
!pip install -U tensorflow-text --quiet
!pip install transformers --quiet
!pip install tensorflow_decision_forests --quiet
!pip install optuna --quiet
!pip install --upgrade pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 33.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.0 MB/s eta 0:00:00
  

In [ ]:
## Library imports
import gzip
import io
import pandas as pd
import warnings
import math
import numpy as np

import tensorflow as tf
from tensorflow import keras
import tensorflow_decision_forests as tfdf
import optuna
from sklearn.metrics import accuracy_score, precision_score, recall_score

import matplotlib.pyplot as plt

from google.colab import files

# Other options
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Data load

In [ ]:
## Load datasets
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
model_checkpoints_path = '/content/drive/My Drive/ds266proj/model_checkpoints'

# Read data from to Google Drive
hs_all_data = pd.read_csv('/content/drive/My Drive/ds266proj/hs_cards_data_text.csv')
nr_all_data = pd.read_csv('/content/drive/My Drive/ds266proj/nr_cards_data_text.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Some summary stats

In [ ]:
## Average number of words per description
nr_desc = list(nr_all_data['description'])
nr_desc_lens = [len(desc.split()) for desc in nr_desc]
print(f'Average NetRunner Description Word Count: {np.array(nr_desc_lens).mean()}')

hs_desc = list(hs_all_data['description'])
hs_desc_lens = [len(desc.split()) for desc in hs_desc]
print(f'Average HearthStone Description Word Count: {np.array(hs_desc_lens).mean()}')

## Card text
nr_texts = [desc.split("The card text says ")[1].strip() for desc in nr_desc]
nr_texts_lens = [len(text.split()) for text in nr_texts]
print(f'Average NetRunner Card Text Word Count: {np.array(nr_texts_lens).mean()}')

hs_texts = [desc.split("The card text says:")[1].strip() for desc in hs_desc]
hs_texts_lens = [len(text.split()) for text in hs_texts]
print(f'Average HearthStone Card Text Word Count: {np.array(hs_texts_lens).mean()}')

## Total unique words in card vocab
hs_split_texts = [text.split() for text in hs_texts]
hs_texts_words = set([word for desc in hs_split_texts for word in desc])
print(f'Unique HearthStone Card Text Word Count: {len(hs_texts_words)}')

nr_split_texts = [text.split() for text in nr_texts]
nr_texts_words = set([word for desc in nr_split_texts for word in desc])
print(f'Unique NetRunner Card Text Word Count: {len(nr_texts_words)}')

Average NetRunner Description Word Count: 55.228450306170515
Average HearthStone Description Word Count: 34.44637873754153
Average NetRunner Card Text Word Count: 27.457842675459254
Average HearthStone Card Text Word Count: 11.282259136212625
Unique HearthStone Card Text Word Count: 2034
Unique NetRunner Card Text Word Count: 1878


## Baseline Model Experimentation

The baseline model will be to use a decision tree and random forest approach that also ignores the text of the cards. This is expected to perform quite poorly.

### Data preparation

In [ ]:
## Set up the various subsetdatasets for modeling

## Include all the mechanics
hs_features_allnontext = hs_all_data[['classes',
                                      # Key info
                                      'health', 'cost', 'attack', 'durability', 'armor',
                                      # Card dscriptors
                                      'Hero', 'Location', 'Minion', 'Spell', 'Weapon',
                                      # tribes
                                      'MECH', 'QUILBOAR', 'DEMON', 'PIRATE', 'TOTEM', 'NAGA', 'ELEMENTAL',
                                      'ALL', 'BEAST', 'MURLOC', 'DRAGON', 'UNDEAD',
                                      # spell schools
                                      'ARCANE', 'FEL', 'FIRE', 'FROST', 'HOLY', 'NATURE', 'SHADOW',
                                      # keywords
                                      'QUICKDRAW', 'COLLECTIONMANAGER_FILTER_MANA_ODD', 'ADJACENT_BUFF', 'GEARS',
                                      'DEAL_DAMAGE', 'FORGETFUL', 'MODULAR', 'SECRET', 'ECHO',
                                      'MULTIPLY_BUFF_VALUE', 'CHOOSE_ONE', 'FORGE', 'WHELP', 'HONORABLE_KILL',
                                      'TWINSPELL', 'QUEST', 'FRENZY', 'OVERHEAL', 'CHARGE', 'FREEZE',
                                      'OVERKILL', 'SPELLPOWER', 'DIVINE_SHIELD', 'AFFECTED_BY_SPELL_POWER',
                                      'ENRAGED', 'COMBO', 'DEATHRATTLE', 'REBORN', 'CORRUPT', 'HIDE_STATS',
                                      'SILENCE', 'TOPDECK', 'POISONOUS', 'WINDFURY', 'TAUNT', 'KABAL',
                                      'COLOSSAL', 'GRIMY_GOONS', 'TRADEABLE', 'OVERLOAD', 'INSPIRE',
                                      'NON_KEYWORD_ECHO', 'SUMMON', 'EXCAVATE', 'AURA', 'BATTLECRY',
                                      'OUTCAST', 'MANATHIRST', 'RUSH', 'HEROPOWER_DAMAGE',
                                      'RECEIVES_DOUBLE_SPELLDAMAGE_BONUS', 'JADE_LOTUS', 'FINALE',
                                      'DEATH_KNIGHT', 'DREDGE', 'INFUSE',
                                      'COLLECTIONMANAGER_FILTER_MANA_EVEN', 'TRIGGER_VISUAL', 'LIFESTEAL',
                                      'START_OF_GAME_KEYWORD', 'IMP', 'RESTORE_HEALTH', 'JADE_GOLEM',
                                      'DISCOVER', 'STEALTH', 'FINISH_ATTACK_SPELL_ON_DAMAGE']].copy()

nr_features_allnontext = nr_all_data[['faction_code',
                                      # Key info
                                      'runner', 'corp', 'ice', 'upgrade', 'hardware', 'resource', 'identity',
                                      'operation', 'agenda', 'program', 'asset', 'event',
                                      # Card dscriptors
                                      'influence', 'unique', 'cost', 'trash_cost', 'memory_cost', 'strength',
                                      'advancement_cost', 'agenda_points', 'base_link',
                                      'influence_limit', 'minimum_deck_size', 'deck_limit',
                                      # # keywords
                                      'morph', 'job', 'location', 'deepnet', 'ambush', 'blackops',
                                      'advertisement', 'lockdown', 'codegate', 'enforcer', 'genetics',
                                      'sabotage', 'chip', 'source', 'grayops', 'consumergrade', 'division',
                                      'priority', 'killer', 'link', 'directive', 'sensie', 'caissa',
                                      'trap', 'remote', 'companion', 'deflector', 'hostile', 'vehicle', 'seedy',
                                      'bioroid', 'terminal', 'harmonic', 'deva', 'political', 'decoder', 'beanstalk',
                                      'psi', 'megacorp', 'cast', 'orgcrime', 'clone', 'reprisal', 'console', 'mythic',
                                      'corporation', 'clan', 'grail', 'ai', 'barrier', 'research', 'character', 'trojan',
                                      'stealth', 'triple', 'icebreaker', 'academic', 'securityprotocol', 'cyborg',
                                      'digital', 'connection', 'transaction', 'ritzy', 'fracter', 'region', 'expendable',
                                      'virus', 'expansion', 'offsite', 'industrial', 'daemon', 'virtual', 'observer',
                                      'facility', 'current', 'alliance', 'cloud', 'executive', 'condition', 'policedepartment',
                                      'weapon', 'destroyer', 'tracer', 'unorthodox', 'natural', 'sentry', 'subsidiary',
                                      'security', 'gear', 'sysop', 'next', 'run', 'double', 'gmod', 'ap',
                                      'cybernetic', 'initiative', 'illicit', 'mod', 'government', 'public']].copy()


In [ ]:
## CODE FOR TRAIN AND TEST SPLITS
##===========================================================

def splitTrainTest(dataset,
                   label,
                   test_ratio = 0.3):

  # Set up the modeling dataset and classification label
  modeling_df = dataset.copy()
  classes = modeling_df[label].unique().tolist()
  print(f"Label classes: {classes}")
  # Convert the label to a numeric value
  modeling_df.loc[:, label] = modeling_df[label].map(classes.index)

  # Split into training and test data
  np.random.seed(2319)
  test_indices = np.random.rand(len(modeling_df)) < test_ratio
  print("{} examples in training, {} examples for testing.".format(
    len(modeling_df[~test_indices]), len(modeling_df[test_indices])))

  return modeling_df[~test_indices], modeling_df[test_indices]

hs_train, hs_test = splitTrainTest(hs_features_allnontext, label = 'classes')
nr_train, nr_test = splitTrainTest(nr_features_allnontext, label = 'faction_code')

Label classes: ['PRIEST', 'WARRIOR', 'WARLOCK', 'SHAMAN', 'DRUID', 'PALADIN', 'MAGE', 'HUNTER', 'ROGUE', 'DEMONHUNTER', 'DEATHKNIGHT', 'NEUTRAL']
5234 examples in training, 2291 examples for testing.
Label classes: ['neutral-corp', 'neutral-runner', 'weyland-consortium', 'nbn', 'jinteki', 'haas-bioroid', 'shaper', 'criminal', 'anarch']
1495 examples in training, 628 examples for testing.


### Decision Tree and Gradient Boosted Tree Modeling

In [ ]:
## CODE TO GENERATE A TREE MODEL USING AN ARBITRARY DATASET
## Basic structure taken from https://www.tensorflow.org/decision_forests/tutorials/beginner_colab
##===========================================================

def fitTreeModel(training_data,
                 test_data,
                 label,
                 tree_type = 'RandomForest',
                 num_trees = 500):
  print("here")
  # Convert into tensorflow dfs
  train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(training_data, label=label)
  test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_data, label=label)
  print("there")

  # Specify the model type
  if tree_type == 'RandomForest':
    model = tfdf.keras.RandomForestModel(num_trees = num_trees)
  elif tree_type == 'GBTree':
    model = tfdf.keras.GradientBoostedTreesModel(num_trees = num_trees)
  else:
    print('Unsupported tree type')
    return

  # Fit the selected model
  model.fit(train_ds)

  # Model evaluation
  model.compile(metrics=["accuracy"])
  evaluation = model.evaluate(test_ds, return_dict=True)

  return model, evaluation


In [ ]:
## Train some models on the input datasets
##--------------------------------------------------
hs_model, hs_model_eval = fitTreeModel(training_data = hs_train,
                                       test_data = hs_test,
                                       label = 'classes')

hs_model_gb, hs_model_eval_gb = fitTreeModel(training_data = hs_train,
                                             test_data = hs_test,
                                             label = 'classes',
                                             tree_type = 'GBTree')

nr_model, nr_model_eval = fitTreeModel(training_data = nr_train,
                                       test_data = nr_test,
                                       label = 'faction_code')

nr_model_gb, nr_model_eval_gb = fitTreeModel(training_data = nr_train,
                                             test_data = nr_test,
                                             label = 'faction_code',
                                             tree_type = 'GBTree')

here
there
Use /tmp/tmpw8z_pw6k as temporary training directory
Reading training dataset...
Training dataset read in 0:00:09.312930. Found 5234 examples.
Training model...
Model trained in 0:00:21.926987
Compiling model...
Model compiled.
3/3 [==============================] - 6s 135ms/step - loss: 0.0000e+00 - accuracy: 0.3483
here
there
Use /tmp/tmp01ckw1p9 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:01.517238. Found 5234 examples.
Training model...
Model trained in 0:01:39.084864
Compiling model...
Model compiled.
3/3 [==============================] - 1s 59ms/step - loss: 0.0000e+00 - accuracy: 0.4107
here
there
Use /tmp/tmpzehoy0qv as temporary training directory
Reading training dataset...
Training dataset read in 0:00:02.162202. Found 1495 examples.
Training model...
Model trained in 0:00:08.097361
Compiling model...
Model compiled.
1/1 [==============================] - 1s 519ms/step - loss: 0.0000e+00 - accuracy: 0.4889
here
there


In [ ]:
## Generate predictions for the validation data:

## HS MODELS
##-----------------------------------------------------------------
## Decision tree
hs_test_tf = tfdf.keras.pd_dataframe_to_tf_dataset(hs_test, label='classes')
hs_model_pred = hs_model.predict(hs_test_tf)
pred_classes_1 = list(np.argmax(hs_model_pred, axis=1))
true_classes_hs = list(hs_test['classes'])

accuracy_1 = accuracy_score(true_classes_hs, pred_classes_1)
precision_1 = precision_score(true_classes_hs, pred_classes_1, average='weighted')
recall_1 = recall_score(true_classes_hs, pred_classes_1, average='weighted')

print("-"*100)
print(f'Hearthstone Tree Model Accuracy: {accuracy_1:.4f}')
print(f'Hearthstone Tree Model Precision: {precision_1:.4f}')
print(f'Hearthstone Tree Model Recall: {recall_1:.4f}')
print("-"*100)

## GB Decision Tree
hs_model_gb_pred = hs_model_gb.predict(hs_test_tf)
pred_classes_2 = list(np.argmax(hs_model_gb_pred, axis=1))

accuracy_2 = accuracy_score(true_classes_hs, pred_classes_2)
precision_2 = precision_score(true_classes_hs, pred_classes_2, average='weighted')
recall_2 = recall_score(true_classes_hs, pred_classes_2, average='weighted')

print(f'Hearthstone Tree Model Accuracy: {accuracy_2:.4f}')
print(f'Hearthstone Tree Model Precision: {precision_2:.4f}')
print(f'Hearthstone Tree Model Recall: {recall_2:.4f}')
print("-"*100)

## NR MODELS
##-----------------------------------------------------------------
## Decision tree
nr_test_tf = tfdf.keras.pd_dataframe_to_tf_dataset(nr_test, label='faction_code')
nr_model_pred = nr_model.predict(nr_test_tf)
pred_classes_3 = list(np.argmax(nr_model_pred, axis=1))
true_classes_nr = list(nr_test['faction_code'])

accuracy_3 = accuracy_score(true_classes_nr, pred_classes_3)
precision_3 = precision_score(true_classes_nr, pred_classes_3, average='weighted')
recall_3 = recall_score(true_classes_nr, pred_classes_3, average='weighted')

print(f'NetRunner Tree Model Accuracy: {accuracy_3:.4f}')
print(f'NetRunner Tree Model Precision: {precision_3:.4f}')
print(f'NetRunner Tree Model Recall: {recall_3:.4f}')
print("-"*100)

## GB Tree
nr_model_gb_pred = nr_model_gb.predict(nr_test_tf)
pred_classes_4 = list(np.argmax(nr_model_gb_pred, axis=1))

accuracy_4 = accuracy_score(true_classes_nr, pred_classes_4)
precision_4 = precision_score(true_classes_nr, pred_classes_4, average='weighted')
recall_4 = recall_score(true_classes_nr, pred_classes_4, average='weighted')

print(f'NetRunner Tree Model Accuracy: {accuracy_4:.4f}')
print(f'NetRunner Tree Model Precision: {precision_4:.4f}')
print(f'NetRunner Tree Model Recall: {recall_4:.4f}')
print("-"*100)

3/3 [==============================] - 1s 134ms/step
----------------------------------------------------------------------------------------------------
Hearthstone Tree Model Accuracy: 0.3483
Hearthstone Tree Model Precision: 0.4746
Hearthstone Tree Model Recall: 0.3483
----------------------------------------------------------------------------------------------------
3/3 [==============================] - 1s 102ms/step
Hearthstone Tree Model Accuracy: 0.4107
Hearthstone Tree Model Precision: 0.4690
Hearthstone Tree Model Recall: 0.4107
----------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 406ms/step
NetRunner Tree Model Accuracy: 0.4889
NetRunner Tree Model Precision: 0.4764
NetRunner Tree Model Recall: 0.4889
----------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 236ms/step
NetRunner Tree Model Accuracy: 0.

#### Neural Nets

In [ ]:
## Input features should be arrays

def genModelArrays(training_data,
                   test_data,
                   label_col):
  # Split the training data
  train_labels = np.array(training_data[label_col])
  train_data = np.array(training_data[training_data.columns[training_data.columns != label_col]])
  # Split the test data
  test_labels = np.array(test_data[label_col])
  test_data = np.array(test_data[test_data.columns[test_data.columns != label_col]])
  # Cast as floats
  train_labels = np.asarray(train_labels).astype('float32')
  train_data = np.asarray(train_data).astype('float32')
  test_labels = np.asarray(test_labels).astype('float32')
  test_data = np.asarray(test_data).astype('float32')

  return train_data, train_labels, test_data, test_labels

# Generate the input sets
hsnn_train_data, hsnn_train_labels, hsnn_test_data, hsnn_test_labels = genModelArrays(hs_train, hs_test, label_col = 'classes')
nrnn_train_data, nrnn_train_labels, nrnn_test_data, nrnn_test_labels = genModelArrays(nr_train, nr_test, label_col = 'faction_code')

print(hsnn_train_data.shape)
print(hsnn_train_labels.shape)

print(nrnn_train_data.shape)
print(nrnn_train_labels.shape)

(5234, 95)
(5234,)
(1495, 125)
(1495,)


In [ ]:
## Code to create a neural net for classification

def buildNNClassifier(n_input_features,
                      hidden_layer_dims,
                      n_classes = 12,
                      hidden_activation = 'relu',
                      dropout_rate = 0.25,
                      learning_rate = 0.001):

  ## Set up the input layer
  input_layer = tf.keras.layers.Input(shape=(n_input_features,), dtype='int32', name = 'input')

  last_hidden_output = input_layer

  ## Set up the hidden layers
  for i, h_dim in enumerate(hidden_layer_dims):
    last_hidden_output = tf.keras.layers.Dense(h_dim, activation=hidden_activation, name='hidden_%d' % i)(last_hidden_output)
    last_hidden_output = tf.keras.layers.Dropout(dropout_rate)(last_hidden_output)

  ## Set up the classification layer
  classification = tf.keras.layers.Dense(12, activation='softmax', name='classification')(last_hidden_output)

  ## Compile the model
  nn_model = tf.keras.models.Model(inputs=input_layer, outputs=[classification])
  nn_model.compile(loss='sparse_categorical_crossentropy',
                   optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                   metrics=['accuracy'])

  return nn_model

In [ ]:
## Neural Net Experiment
##========================================================

## Using the optuna package to search for most performant NN model

def objectiveHS(trial):
  # First create the search space
  #----------------------------------------
  # Try activation functions
  activation = trial.suggest_categorical("hidden_activation", ["sigmoid", "relu"])
  # Try numbers of layers
  num_hidden_layers = trial.suggest_int("num_hidden_layers", 1, 3)
  # For the hidden layers try various layer sizes
  hidden_layer_dims = []
  for layer_n in range(num_hidden_layers):
    hidden_size = trial.suggest_int("hidden_size", 10, 20, step=1)
    hidden_layer_dims.append(hidden_size)
  trial.set_user_attr('hidden_layer_dims', hidden_layer_dims)
  # Try some number of epochs
  num_epochs = trial.suggest_int("num_epochs", 10, 25, step=1)
  # Next set up the model, run it, and check the finalized accuracy
  #----------------------------------------
  trial_model = buildNNClassifier(n_input_features=hsnn_train_data.shape[1],
                                  hidden_layer_dims=hidden_layer_dims,
                                  hidden_activation = activation,
                                  learning_rate = 0.001)
  trial_model_history = trial_model.fit(hsnn_train_data,
                                        hsnn_train_labels,
                                        validation_data = (hsnn_test_data, hsnn_test_labels),
                                        epochs = num_epochs,
                                        verbose = 0)
  # Save model metrics
  trial_pred = trial_model.predict(hsnn_test_data)
  pred_classes = list(np.argmax(trial_pred, axis=1))

  accuracy = accuracy_score(true_classes_hs, pred_classes)
  precision = precision_score(true_classes_hs, pred_classes, average='weighted')
  recall = recall_score(true_classes_hs, pred_classes, average='weighted')

  trial.set_user_attr('accuracy', accuracy)
  trial.set_user_attr('precision', precision)
  trial.set_user_attr('recall', recall)

  # Finally define the thing to minimize, in this case 1 minus the model validation accuracy
  #------------------------------------------
  result = 1 - trial_model_history.history['val_accuracy'][-1]

  return result

def objectiveNR(trial):
  # First create the search space
  #----------------------------------------
  # Try activation functions
  activation = trial.suggest_categorical("hidden_activation", ["sigmoid", "relu"])
  # Try numbers of layers
  num_hidden_layers = trial.suggest_int("num_hidden_layers", 1, 3)
  # For the hidden layers try various layer sizes
  hidden_layer_dims = []
  for layer_n in range(num_hidden_layers):
    hidden_size = trial.suggest_int("hidden_size", 10, 20, step=1)
    hidden_layer_dims.append(hidden_size)
  trial.set_user_attr('hidden_layer_dims', hidden_layer_dims)
  # Try some number of epochs
  num_epochs = trial.suggest_int("num_epochs", 10, 25, step=1)
  # Next set up the model, run it, and check the finalized accuracy
  #----------------------------------------
  trial_model = buildNNClassifier(n_input_features=nrnn_train_data.shape[1],
                                  hidden_layer_dims=hidden_layer_dims,
                                  hidden_activation = activation,
                                  learning_rate = 0.001)
  trial_model_history = trial_model.fit(nrnn_train_data,
                                        nrnn_train_labels,
                                        validation_data = (nrnn_test_data, nrnn_test_labels),
                                        epochs = num_epochs,
                                        verbose = 0)
  # Save model metrics
  trial_pred = trial_model.predict(nrnn_test_data)
  pred_classes = list(np.argmax(trial_pred, axis=1))

  accuracy = accuracy_score(true_classes_nr, pred_classes)
  precision = precision_score(true_classes_nr, pred_classes, average='weighted')
  recall = recall_score(true_classes_nr, pred_classes, average='weighted')

  trial.set_user_attr('accuracy', accuracy)
  trial.set_user_attr('precision', precision)
  trial.set_user_attr('recall', recall)

  # Finally define the thing to minimize, in this case 1 minus the model validation accuracy
  #------------------------------------------
  result = 1 - trial_model_history.history['val_accuracy'][-1]

  return result

In [ ]:
hsnn_experiment = optuna.create_study(study_name="HSNN_Experiment")
hsnn_experiment.optimize(objectiveHS, n_trials=100)

[I 2024-04-11 19:13:06,112] A new study created in memory with name: HSNN_Experiment


72/72 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2024-04-11 19:13:32,471] Trial 0 finished with value: 0.7001309394836426 and parameters: {'hidden_activation': 'sigmoid', 'num_hidden_layers': 1, 'hidden_size': 16, 'num_epochs': 14}. Best is trial 0 with value: 0.7001309394836426.


72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2024-04-11 19:14:09,056] Trial 1 finished with value: 0.6713225543498993 and parameters: {'hidden_activation': 'relu', 'num_hidden_layers': 2, 'hidden_size': 14, 'num_epochs': 22}. Best is trial 1 with value: 0.6713225543498993.


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2024-04-11 19:44:20,976] Trial 97 finished with value: 0.6621562540531158 and parameters: {'hidden_activation': 'relu', 'num_hidden_layers': 1, 'hidden_size': 17, 'num_epochs': 19}. Best is trial 32 with value: 0.6390222609043121.


72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


[I 2024-04-11 19:44:36,263] Trial 98 finished with value: 0.6534264385700226 and parameters: {'hidden_activation': 'relu', 'num_hidden_layers': 1, 'hidden_size': 16, 'num_epochs': 18}. Best is trial 32 with value: 0.6390222609043121.


72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


[I 2024-04-11 19:44:52,335] Trial 99 finished with value: 0.6460061073303223 and parameters: {'hidden_activation': 'relu', 'num_hidden_layers': 1, 'hidden_size': 16, 'num_epochs': 18}. Best is trial 32 with value: 0.6390222609043121.


In [ ]:
print(hsnn_experiment.best_trial.params)
print(hsnn_experiment.best_trial.user_attrs)
print(f"Validation Acc: {1 - hsnn_experiment.best_trial.values[0]}")


{'hidden_activation': 'relu', 'num_hidden_layers': 1, 'hidden_size': 20, 'num_epochs': 20}
{'hidden_layer_dims': [20], 'accuracy': 0.36097773897861196, 'precision': 0.4376747138091939, 'recall': 0.36097773897861196}
Validation Acc: 0.36097773909568787


In [ ]:
nrnn_experiment = optuna.create_study(study_name="NRNN_Experiment")
nrnn_experiment.optimize(objectiveNR, n_trials=100)

[I 2024-04-11 20:01:29,003] A new study created in memory with name: NRNN_Experiment


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2024-04-11 20:01:52,732] Trial 0 finished with value: 0.7643312066793442 and parameters: {'hidden_activation': 'sigmoid', 'num_hidden_layers': 3, 'hidden_size': 15, 'num_epochs': 18}. Best is trial 0 with value: 0.7643312066793442.


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2024-04-11 20:02:10,431] Trial 1 finished with value: 0.6257961690425873 and parameters: {'hidden_activation': 'relu', 'num_hidden_layers': 2, 'hidden_size': 12, 'num_epochs': 11}. Best is trial 1 with value: 0.6257961690425873.


...

[I 2024-04-11 20:24:15,682] Trial 97 finished with value: 0.5318471193313599 and parameters: {'hidden_activation': 'relu', 'num_hidden_layers': 1, 'hidden_size': 18, 'num_epochs': 24}. Best is trial 69 with value: 0.5111464858055115.


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2024-04-11 20:24:31,006] Trial 98 finished with value: 0.5987261235713959 and parameters: {'hidden_activation': 'sigmoid', 'num_hidden_layers': 1, 'hidden_size': 19, 'num_epochs': 23}. Best is trial 69 with value: 0.5111464858055115.


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


[I 2024-04-11 20:24:46,887] Trial 99 finished with value: 0.5350318551063538 and parameters: {'hidden_activation': 'relu', 'num_hidden_layers': 1, 'hidden_size': 18, 'num_epochs': 25}. Best is trial 69 with value: 0.5111464858055115.


In [ ]:
print(nrnn_experiment.best_trial.params)
print(nrnn_experiment.best_trial.user_attrs)
print(f"Validation Acc: {1 - nrnn_experiment.best_trial.values[0]}")

{'hidden_activation': 'relu', 'num_hidden_layers': 1, 'hidden_size': 20, 'num_epochs': 18}
{'hidden_layer_dims': [20], 'accuracy': 0.4888535031847134, 'precision': 0.475513944938763, 'recall': 0.4888535031847134}
Validation Acc: 0.4888535141944885
